In [9]:
import matplotlib.pyplot as plt
from Objects.Simulation import Simulation
import numpy as np
import pandas as pd
%pylab
%matplotlib inline



Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [10]:
%pylab
%matplotlib inline

from matplotlib import lines, markers
from cycler import cycler

# Create cycler object. Use any styling from above you please
monochromeWithMarker = (cycler('color', ['k']) * cycler('linestyle', ['-', '--', ':', '-.']) * cycler('marker', [ '.' ,'^']))
monochromeNoMarker = (cycler('color', ['k']) * cycler('linestyle', ['-', '--', ':', '-.']) * cycler('marker', [',']))
monochrome = monochromeNoMarker.concat(monochromeWithMarker)

plt.set_cmap('gray')
# Overriding styles for current script
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['axes.spines.left'] = False
plt.rcParams['figure.figsize'] = (12,5)
mpl.rcParams['figure.dpi']= 300
plt.rcParams['axes.prop_cycle'] = monochrome

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


<Figure size 432x288 with 0 Axes>

In [11]:
def cal_one_p(p):
    return np.mean(p.delays), np.max(p.delays)

def cal_vdr(ps, tg):
    if type(ps) is not list:
        ps = [ps]
    total_vehPass = 0
    total_tg = 0
    for p in ps:
        total_vehPass += p.vehPass
        total_tg += tg
    return total_vehPass / total_tg * 3600

def get_data_array(p,tg):
    delaymean, delaymax = cal_one_p(p)
    return [delaymean, delaymax, cal_vdr(p, tg)]

In [20]:
simulation_time = 60 # seconds
n = 100 # number of vehicles
intended_speed = 80 * 1000/3600 # intended speed / speed limit of the road
sim = Simulation(time = simulation_time, avStep = 100)


LRT = [1.4, -1.4]
HSR = [0.50, -0.50]

human_profile = [1.7, -2.0]
av_profile = HSR

paras={
    'v_intend': intended_speed,
    'human_max_acc': human_profile[0],
    'human_max_dec': human_profile[1],
    'max_acc': av_profile[0],
    'max_dec': av_profile[1],
    'tsys': 1.0,
    'human_T': 1.8,
    'AV_T': 1.6
}

human = sim.run_human_simulation(n, paras)
get_data_array(human, simulation_time)

[21.045918298965475, 39.95519946607026, 1800.0]

In [13]:
human_ps = []
av_ps = []
cav_ps = []
start_acc = 1.4
end_acc = 0.50
start_dec = -1.4
end_dec = -0.50
totalLoop = int((start_acc - end_acc-0.0001) / 0.04 + 1) + 1
secondLoop = int((end_dec - start_dec -0.0001) / 0.04 + 1) + 1
print(totalLoop, secondLoop)
xs_acc = [round(x,2) for x in [start_acc - i * 0.04 for i in range(totalLoop)]]
ys_dec = [round(x,2) for x in [start_dec + j * 0.04 for j in range(secondLoop)]]

24 24


In [14]:
av_avg_delay = pd.DataFrame(index = ys_dec, columns=xs_acc)
av_max_delay = pd.DataFrame(index = ys_dec, columns=xs_acc)
av_vdr = pd.DataFrame(index = ys_dec, columns=xs_acc)
cav_avg_delay = pd.DataFrame(index = ys_dec, columns=xs_acc)
cav_max_delay = pd.DataFrame(index = ys_dec, columns=xs_acc)
cav_vdr = pd.DataFrame(index = ys_dec, columns=xs_acc)


for col in xs_acc:
    cur_acc = col
    for idx in ys_dec:
        cur_dec = idx
        paras['max_acc'] = cur_acc
        paras['max_dec'] = cur_dec
        print(cur_acc,cur_dec)
        av = sim.run_av_simulation(n, paras)
        cav = sim.run_cav_simulation(n, paras)
        av=get_data_array(av, simulation_time)
        cav=get_data_array(cav, simulation_time)
        print(cav)
        av_avg_delay.at[idx, col] = av[0]
        av_max_delay.at[idx, col] = av[1]
        av_vdr.at[idx, col] = av[2]
        cav_avg_delay.at[idx, col] = cav[0]
        cav_max_delay.at[idx, col] = cav[1]
        cav_vdr.at[idx, col] = cav[2]

1.4 -1.4
[9.306763431451888, 17.90977114805537, 3240.0]
1.4 -1.36
[9.306763431448045, 17.909771147983726, 3240.0]
1.4 -1.32
[9.306763431443867, 17.90977114790607, 3240.0]
1.4 -1.28
[9.306763431438958, 17.909771147815665, 3240.0]
1.4 -1.24
[9.30676343143303, 17.90977114770818, 3240.0]
1.4 -1.2
[9.30676343142621, 17.909771147585506, 3240.0]
1.4 -1.16
[9.306763431418641, 17.909771147450055, 3240.0]
1.4 -1.12
[9.306763431409022, 17.909771147280928, 3240.0]
1.4 -1.08
[9.306763431398142, 17.90977114709068, 3240.0]
1.4 -1.04
[9.30676343138591, 17.909771146878395, 3240.0]
1.4 -1.0
[9.306763431370614, 17.9097711466171, 3240.0]
1.4 -0.96
[9.306763431352858, 17.90977114631577, 3240.0]
1.4 -0.92
[9.30676343133243, 17.909771145972474, 3240.0]
1.4 -0.88
[9.306763431307779, 17.90977114556301, 3240.0]
1.4 -0.84
[9.306763431279087, 17.909771145089817, 3240.0]
1.4 -0.8
[9.306763431244136, 17.909771144521724, 3240.0]
1.4 -0.76
[9.306763431204232, 17.909771143876615, 3240.0]
1.4 -0.72
[9.306763431154726, 

[9.613410293568995, 18.138194589809846, 3180.0]
1.2 -0.48
[9.613410293494614, 18.138194588658962, 3180.0]
1.16 -1.4
[9.56389103506602, 17.89309391208869, 3120.0]
1.16 -1.36
[9.563891035065438, 17.89309391207675, 3120.0]
1.16 -1.32
[9.563891035064739, 17.893093912062483, 3120.0]
1.16 -1.28
[9.56389103506399, 17.893093912047263, 3120.0]
1.16 -1.24
[9.563891035062955, 17.8930939120269, 3120.0]
1.16 -1.2
[9.563891035061769, 17.893093912003483, 3120.0]
1.16 -1.16
[9.563891035060404, 17.893093911976976, 3120.0]
1.16 -1.12
[9.56389103505863, 17.893093911943136, 3120.0]
1.16 -1.08
[9.563891035056468, 17.893093911902287, 3120.0]
1.16 -1.04
[9.563891035054052, 17.89309391185703, 3120.0]
1.16 -1.0
[9.563891035050773, 17.89309391179675, 3120.0]
1.16 -0.96
[9.563891035047009, 17.893093911728087, 3120.0]
1.16 -0.92
[9.563891035042335, 17.893093911644144, 3120.0]
1.16 -0.88
[9.563891035036594, 17.893093911542156, 3120.0]
1.16 -0.84
[9.563891035029481, 17.893093911417537, 3120.0]
1.16 -0.8
[9.56389103

[10.126681181144669, 18.454003459569606, 3060.0]
0.96 -0.6
[10.12668118113511, 18.454003459406557, 3060.0]
0.96 -0.56
[10.1266811811229, 18.454003459201413, 3060.0]
0.96 -0.52
[10.126681181106587, 18.45400345893166, 3060.0]
0.96 -0.48
[10.126681181085491, 18.45400345858856, 3060.0]
0.92 -1.4
[10.313806533443415, 18.700386062110724, 3060.0]
0.92 -1.36
[10.31380653344334, 18.700386062109065, 3060.0]
0.92 -1.32
[10.313806533443252, 18.700386062107043, 3060.0]
0.92 -1.28
[10.313806533443135, 18.70038606210441, 3060.0]
0.92 -1.24
[10.31380653344299, 18.70038606210121, 3060.0]
0.92 -1.2
[10.313806533442813, 18.700386062097255, 3060.0]
0.92 -1.16
[10.31380653344258, 18.70038606209227, 3060.0]
0.92 -1.12
[10.313806533442296, 18.70038606208617, 3060.0]
0.92 -1.08
[10.313806533441928, 18.700386062078554, 3060.0]
0.92 -1.04
[10.313806533441443, 18.70038606206861, 3060.0]
0.92 -1.0
[10.313806533440875, 18.700386062057017, 3060.0]
0.92 -0.96
[10.313806533440061, 18.70038606204093, 3060.0]
0.92 -0.9

[11.077408500402212, 19.143236924437293, 2880.0]
0.72 -0.8
[11.077408500402084, 19.143236924434557, 2880.0]
0.72 -0.76
[11.077408500401901, 19.143236924430745, 2880.0]
0.72 -0.72
[11.077408500401647, 19.143236924425505, 2880.0]
0.72 -0.68
[11.07740850040127, 19.143236924417998, 2880.0]
0.72 -0.64
[11.077408500400724, 19.143236924407315, 2880.0]
0.72 -0.6
[11.077408500399963, 19.143236924392763, 2880.0]
0.72 -0.56
[11.077408500398802, 19.14323692437102, 2880.0]
0.72 -0.52
[11.077408500397182, 19.14323692434131, 2880.0]
0.72 -0.48
[11.077408500394794, 19.143236924298485, 2880.0]
0.68 -1.4
[11.256623522125498, 19.188179832325588, 2820.0]
0.68 -1.36
[11.256623522125498, 19.188179832325588, 2820.0]
0.68 -1.32
[11.256623522125498, 19.18817983232558, 2820.0]
0.68 -1.28
[11.256623522125498, 19.18817983232557, 2820.0]
0.68 -1.24
[11.256623522125498, 19.18817983232557, 2820.0]
0.68 -1.2
[11.256623522125498, 19.18817983232556, 2820.0]
0.68 -1.16
[11.256623522125496, 19.188179832325538, 2820.0]
0.

[13.293137571866538, 21.03054549298337, 2580.0]
0.48 -1.0
[13.293137571866538, 21.03054549298337, 2580.0]
0.48 -0.96
[13.293137571866538, 21.03054549298337, 2580.0]
0.48 -0.92
[13.293137571866538, 21.03054549298337, 2580.0]
0.48 -0.88
[13.293137571866538, 21.03054549298337, 2580.0]
0.48 -0.84
[13.293137571866538, 21.03054549298337, 2580.0]
0.48 -0.8
[13.293137571866538, 21.03054549298337, 2580.0]
0.48 -0.76
[13.293137571866538, 21.03054549298337, 2580.0]
0.48 -0.72
[13.293137571866538, 21.030545492983357, 2580.0]
0.48 -0.68
[13.293137571866538, 21.030545492983357, 2580.0]
0.48 -0.64
[13.293137571866536, 21.030545492983318, 2580.0]
0.48 -0.6
[13.293137571866533, 21.03054549298324, 2580.0]
0.48 -0.56
[13.29313757186653, 21.03054549298317, 2580.0]
0.48 -0.52
[13.29313757186652, 21.03054549298298, 2580.0]
0.48 -0.48
[13.293137571866504, 21.030545492982576, 2580.0]


In [15]:
av_avg_delay

,1.4,1.36,1.32,1.28,1.24,1.2,1.16,1.12,1.08,1.04,...,0.84,0.8,0.76,0.72,0.68,0.64,0.6,0.56,0.52,0.48
-1.40,20.969,21.1069,20.683,20.8384,21.0056,21.1859,21.3809,21.0062,21.2316,21.4778,...,21.8903,22.3269,22.8206,23.383,23.357,24.0875,24.9441,25.2258,26.4181,27.8808
-1.36,20.938,21.0752,21.2221,20.8063,20.9726,21.152,21.346,20.9716,21.1959,21.4408,...,22.4713,22.2829,22.774,23.3335,23.3066,24.0333,24.8853,25.1648,26.3508,27.8055
-1.32,20.9058,21.0422,21.1884,20.773,20.9384,21.1169,21.3099,20.9358,21.1589,21.4025,...,22.4261,22.2373,22.7259,23.2824,23.2545,23.9773,24.8246,25.1017,26.2812,27.7278
-1.28,20.8723,21.008,21.1533,20.7384,20.903,21.0804,21.2724,21.4806,21.1205,21.3628,...,22.3792,22.1902,22.676,23.2294,23.2006,23.9193,24.7619,25.0366,26.2093,27.6475
-1.24,20.8376,20.9725,21.1169,21.272,20.8661,21.0426,21.2334,21.4404,21.0807,21.3216,...,22.3306,22.1413,22.6244,23.1746,23.1447,23.8593,24.6969,24.9692,26.135,27.5644
-1.20,20.8014,20.9354,21.079,21.2332,20.8277,21.0032,21.1929,21.3986,21.0392,21.2787,...,22.2802,22.0905,22.5708,23.1177,23.0868,23.7971,24.6296,24.8993,26.058,27.4783
-1.16,20.7637,20.8969,21.0396,21.1927,21.3575,20.9621,21.1507,21.3552,21.5777,21.2341,...,22.2278,22.6599,22.5151,23.0587,23.6828,23.7326,24.5598,24.8269,25.9781,27.3892
-1.12,21.285,20.8566,20.9984,21.1505,21.3143,20.9193,21.1067,21.3098,21.5309,21.1877,...,22.1733,22.6026,22.4573,22.9973,23.6174,23.6656,24.4873,24.7517,25.8953,27.2967
-1.08,21.2423,20.8145,20.9553,21.1064,21.269,20.8746,21.0607,21.2625,21.4821,21.1392,...,22.1165,22.5429,22.397,22.9334,23.5492,23.5959,24.412,25.3804,25.8092,27.2006
-1.04,21.1976,21.3305,20.9103,21.0603,21.2217,21.3959,21.0126,21.213,21.4311,21.6692,...,22.0572,22.4806,22.3341,22.8668,23.4782,23.5232,24.3335,25.2949,25.7196,27.1007


In [16]:
subFolder = "SG1/"
av_avg_delay.to_csv(subFolder+'SG1_av_avg_delay.csv')
av_max_delay.to_csv(subFolder+'SG1_av_max_delay.csv')
av_vdr.to_csv(subFolder+'SG1_av_vdr.csv')
cav_avg_delay.to_csv(subFolder+'SG1_cav_avg_delay.csv')
cav_max_delay.to_csv(subFolder+'SG1_cav_max_delay.csv')
cav_vdr.to_csv(subFolder+'SG1_cav_vdr.csv')